# **Web and Social Media Analysis**
**Assignment 1**

Name: Saleem El Bouri

ID: 20228086

In [1]:
#Install libraries

!pip install BeautifulSoup4
!pip install requests

In [4]:
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
import re

base_url = 'https://psut.edu.jo/'

base_response = requests.get(base_url)


if base_response.status_code == 200:
    base_html_content = base_response.text

else:
    print(f"Failed to fetch the webpage. Status code: {base_response.status_code}")

base_soup = BeautifulSoup(base_html_content, 'html.parser')

In [5]:
scools_full_url =[]

main_menu = base_soup.find('nav',class_='head_btm_menu')

academics = main_menu.find('a',string='Academics')

if academics:
    # Navigate to the sibling <ul> tag of the "Academics" link and find all <a> tags within it
    schools_links = academics.find_next_sibling('ul').find_all('a')

    # Extract the href attribute values of the school links
    schools_hrefs = set([link.get('href') for link in schools_links])
    print(schools_hrefs)

    schools_url = []

    for i in schools_hrefs:

      if i.startswith('/en/school/'):
        print(i)
        joined_url = urljoin(base_url, i)
        schools_url.append(joined_url)




    print(schools_url)
else:
    print('Not Found')


{'/en/ktsbt-contact-us', '/en/about-ktsbt', '/en/school/School_of_Engineering', '/en/school/school_of_computing_sciences', '/en/it-contact-us', '/en/about-king-abdullah-ii-school-of-engineering', '/en/school/King_business_technology', '/en/contact-us-3', '/en/about-khscs', '/en/school/School_of_Graduate_Studies_Scientific_Research'}
/en/school/School_of_Engineering
/en/school/school_of_computing_sciences
/en/school/King_business_technology
/en/school/School_of_Graduate_Studies_Scientific_Research
['https://psut.edu.jo/en/school/School_of_Engineering', 'https://psut.edu.jo/en/school/school_of_computing_sciences', 'https://psut.edu.jo/en/school/King_business_technology', 'https://psut.edu.jo/en/school/School_of_Graduate_Studies_Scientific_Research']


In [99]:
# we will iterate through each URL
school_info = []
PSUT_Staff_info = []
#staff_name_w_position =[]

for SURL in schools_url:
  url2 = SURL

  response2 = requests.get(url2)
  if response2.status_code == 200:
      html_content2 = response2.text

  else:
      print(f"Failed to fetch the webpage. Status code: {response2.status_code}")

  soup2 = BeautifulSoup(html_content2, 'html.parser')

  #Scraping schools names, programs
  school = soup2.find('h2')
  school_name = school.text.strip()

# I put a condition to check whether the school name was the
# King Abdullah I  School of Graduate Studies & Scientific Research
# It has a different page structure than the rest of schools
  #school_info = []

  if school_name != 'King Abdullah I  School of Graduate Studies & Scientific Research':

    print('condition met\n\n')

    print(f'-----------------------Starting With {school_name} ----------------------------------------------\n\n')


    dean_html = soup2.find('div',class_='meetfaculty_single')

    #Extracting Dean Name

    dean_name = dean_html.find('h4').text.strip()

    #Extracting Programs from Each School
    programs_tab = soup2.find('div',  id='nav-home')

    programs = programs_tab.find_all('div',class_='graduates-tabbing-box')

    bachelors_programs = []
    masters_programs = []
    all_programs = []

    for program in programs:
        program_name = program.find('h4').text.strip()
        if "Bachelor" in program_name:
            bachelors_programs.append(program_name)
        elif "Master" in program_name:
            masters_programs.append(program_name)

    all_programs = bachelors_programs + masters_programs

    #print(school_name)
    #print("Dean: ",members_names)
    #print("Bachelor's Programs: ", bachelors_programs)
    #print("Master's Programs: ", masters_programs)




    dept_tab = soup2.find('div',  id='nav-profile')

    depts = dept_tab.find_all('div',class_='graduates-tabbing-box')
    #programe_links = programs.find_all('h4')
    dept_name = [link.find('h4').text.strip() for link in depts]


    keyword = ['all-staff']
    full_url =[]

    #get the academic and administrative staff URLs to automate the extraction in the next part

    for link in soup2.find_all('a',attrs={'href': re.compile("^/")}):

        href = link.get('href')
        #print(href)
        if any(word in href for word in keyword):

            joined_url = urljoin(url2, href)


            full_url.append(joined_url)


    #print(full_url)

    # Extracting all academic and administrative staff info from each school

    for link in full_url:

        #print(link)

        response3 = requests.get(link)
        if response3.status_code == 200:
          html_content3 = response3.text
        else:
          print(f"Failed to fetch the webpage. Status code: {response3.status_code}")

        soup3 = BeautifulSoup(html_content3,'html.parser')
        out = []
        #staff_name_w_position =[]
        #print(len(soup3.find_all('li', class_='page-item')))

        #There are pages in the school's staff list so i am getting the page's links to scrape through them

        for i in range(1,5):

          joinedURL = urljoin(link, f'?page={i}')
          staff_page_response = requests.get(joinedURL)
          if staff_page_response.status_code == 200:
            staff_page_html_content = staff_page_response.text
          else:
            print(f"Failed to fetch the webpage. Status code: {staff_page_response.status_code}")
            break;


          staff_page_soup = BeautifulSoup(staff_page_html_content,'html.parser')

          #got the HTML part that contains staff names, degree and position

          out = staff_page_soup.find_all('div',class_='member-trustees')

          for i in out:


            # Extracted the names alone
            name = i.find('h4').text.strip()

            #Extracted the positions with degree alone ..... will be separated again later below
            position_w_degree = i.find('p')

            # getting staff specific available info

            staff_URL = i.find('a', attrs={'href': re.compile("^/")})

            staff_href = staff_URL.get('href')
            #print(href)
            staff_joined_url = urljoin(link, staff_href)

            staff_response = requests.get(staff_joined_url)
            if staff_response.status_code == 200:
              staff_html_content = staff_response.text
            else:
              print(f"Failed to fetch the webpage. Status code: {staff_response.status_code}")
              break;

            staff_info_soup = BeautifulSoup(staff_html_content,'html.parser')

            # Extracting staff summary

            summary_html = staff_info_soup.find('div',class_='col-md-6 researcher-bio')
            summary_final = summary_html.find('p').text.strip()
            summary_final_finalest = summary_final.replace('\n','')
            summary_final_finalestest = summary_final_finalest.replace('_','')


            # Extracting Available Personal and contact Info

            staff_contact_info = staff_info_soup.find('div',class_='mt-3')
            staff_tel_and_email = staff_contact_info.find_all('p')
            final_contact = [cont.text.strip() for cont in staff_tel_and_email]

            telephone =''
            email = ''

            for item in final_contact:
              if item.startswith('Telephone'):
                telephone = item.split(': ', 1)[1]
                #print(telephone)
              elif item.startswith('Email'):
                email = item.split(': ')[1]
                #print(email)


            degree = ''

            if position_w_degree.find('br'):
              degree   =  position_w_degree.find('br').next_sibling.strip()
              position =  position_w_degree.text.split(degree)[0].strip()

            else:
              position = ''
              degree = position_w_degree.text.strip()



            staff_name_w_position.append({'Name': name,
                                          'School': school_name,
                                          'Position_1': degree,
                                          'Position_2': position,
                                          'School': school_name,
                                          'Summary':summary_final_finalestest,
                                          'Telephone and EXT': telephone,
                                          'Email':email})


          #prof_name.append([name.find('h4').text.strip() for name in out])

        #The output had too many problems and duplicates as it was difficult to handle navigating through pages so they are handled here

        staff_no_dups = set(tuple(sorted(d.items())) for d in staff_name_w_position)

        #The final output for the staff is here
        final_staff =  [dict(t) for t in staff_no_dups]

        names_alone = []

        for i in final_staff:
          namefdic = i.get("Name")
          if namefdic is not None:
            names_alone.append(namefdic)


        #Extracting school summary and contact info


        #school_info.append({"School Name": school_name ,
         #                   "Dean": dean_name,
          #                  "URL": url2,
           #                 "Departments": dept_name ,
            #                "Prgorams": all_programs ,
             #               "Staff" : names_alone})
        #PSUT_Schools_info.append(school_info)
        #PSUT_Staff_info.append(final_staff)

        #staff_directory_per_school = []

        print('\n')

        #staff_directory_per_school.append({"School Name": school_name,
        #                                  "Staff": final_staff})


        #print(school_info)

        #print(final_staff)

        #print(school_info)

        #print(staff_directory_per_school)

  elif school_name == 'King Abdullah I  School of Graduate Studies & Scientific Research':

    print('condition not met\n\n\n')

    print(f'-----------------------Starting With {school_name} ----------------------------------------------\n\n')

    # Extracting dean's name

    Name_of_dean = soup2.find('strong').get_text()
    Name_of_scie = Name_of_dean.split('Research')[-1]

    keyword = ['all-staff']
    full_url =[]

    #get the academic and administrative staff URLs to automate the extraction in the next part

    for link in soup2.find_all('a',attrs={'href': re.compile("^/")}):

        href = link.get('href')
        #print(href)
        if any(word in href for word in keyword):

            joined_url = urljoin(url2, href)


            full_url.append(joined_url)


    #print(full_url)

    # Extracting all academic and administrative staff info from each school

    for link in full_url:

        #print(link)

        response3 = requests.get(link)
        if response3.status_code == 200:
          html_content3 = response3.text
        else:
          print(f"Failed to fetch the webpage. Status code: {response3.status_code}")

        soup3 = BeautifulSoup(html_content3,'html.parser')
        out = []
        #staff_name_w_position =[]
        #print(len(soup3.find_all('li', class_='page-item')))

        for i in range(1,5):

          joinedURL = urljoin(link, f'?page={i}')
          staff_page_response = requests.get(joinedURL)
          if staff_page_response.status_code == 200:
            staff_page_html_content = staff_page_response.text
          else:
            print(f"Failed to fetch the webpage. Status code: {staff_page_response.status_code}")
            break;


          staff_page_soup = BeautifulSoup(staff_page_html_content,'html.parser')

          #got the HTML part that contains staff names, degree and position

          out = staff_page_soup.find_all('div',class_='member-trustees')

          for i in out:


            # Extracted the names alone
            name = i.find('h4').text.strip()

            #Extracted the positions with degree alone ..... will be separated again later below
            position_w_degree = i.find('p')

            # getting staff specific available info

            staff_URL = i.find('a', attrs={'href': re.compile("^/")})

            staff_href = staff_URL.get('href')
            #print(href)
            staff_joined_url = urljoin(link, staff_href)

            staff_response = requests.get(staff_joined_url)
            if staff_response.status_code == 200:
              staff_html_content = staff_response.text
            else:
              print(f"Failed to fetch the webpage. Status code: {staff_response.status_code}")
              break;

            staff_info_soup = BeautifulSoup(staff_html_content,'html.parser')

            # Extracting staff summary

            summary_html = staff_info_soup.find('div',class_='col-md-6 researcher-bio')
            summary_final = summary_html.find('p').text.strip()
            summary_final_finalest = summary_final.replace('\n','')
            summary_final_finalestest = summary_final_finalest.replace('_','')


            # Extracting Available Personal and contact Info

            staff_contact_info = staff_info_soup.find('div',class_='mt-3')
            staff_tel_and_email = staff_contact_info.find_all('p')
            final_contact = [cont.text.strip() for cont in staff_tel_and_email]

            telephone =''
            email = ''

            for item in final_contact:
              if item.startswith('Telephone'):
                telephone = item.split(': ', 1)[1]
                #print(telephone)
              elif item.startswith('Email'):
                email = item.split(': ')[1]
                #print(email)


            degree = ''

            if position_w_degree.find('br'):
              degree   =  position_w_degree.find('br').next_sibling.strip()
              position =  position_w_degree.text.split(degree)[0].strip()

            else:
              position = ''
              degree = position_w_degree.text.strip()



            staff_name_w_position.append({'Name': name,
                                          'School': school_name,
                                          'Position_1': degree,
                                          'Position_2': position,
                                          'School': school_name,
                                          'Summary':summary_final_finalestest,
                                          'Telephone and EXT': telephone,
                                          'Email':email})


          #prof_name.append([name.find('h4').text.strip() for name in out])

        #The output had too many problems and duplicates as it was difficult to handle navigating through pages so they are handled here

        staff_no_dups = set(tuple(sorted(d.items())) for d in staff_name_w_position)

        #The final output for the staff is here
        final_staff =  [dict(t) for t in staff_no_dups]

        names_alone = []

        for i in final_staff:
          namefdic = i.get("Name")
          if namefdic is not None:
            names_alone.append(namefdic)


    # Exctracting Programs

    programs_tab = soup2.find('div',  id='nav-home')

    programs = programs_tab.find_all('div',class_='graduates-tabbing-box')

    bachelors_programs = []
    masters_programs = []
    all_programs = []

    for program in programs:
        program_name = program.find('h4').text.strip()
        if "Bachelor" in program_name:
            bachelors_programs.append(program_name)
        elif "Master" in program_name:
            masters_programs.append(program_name)
        else:
            masters_programs.append(program_name)

    all_programs = bachelors_programs + masters_programs

    # Extracting Fields of Research

    scie_research_tab = soup2.find('div',  class_='row aboutgraduate trainingworkshop-box mt-1 school-custom-color')


    depts = scie_research_tab.find_all('div',class_='related-items')

    #programe_links = programs.find_all('h4')
    dept_name = [link.find('h2').text.strip() for link in depts]

    #school_info.append({"School Name": school_name ,
     #                       "Dean": Name_of_scie,
      #                      "URL": url2,
       #                     "Research Departments": dept_name ,
        #                    "Prgorams": all_programs ,
         #                   "Staff" : names_alone})
    #PSUT_Schools_info.append(school_info)
    #PSUT_Staff_info.append(final_staff)


  if school_name != 'King Abdullah I  School of Graduate Studies & Scientific Research':



    school_info.append({"School Name": school_name ,
                        "Dean": dean_name,
                        "URL": url2,
                        "Departments": dept_name ,
                        "Prgorams": all_programs })
  else:

    school_info.append({"School Name": school_name ,
                        "Dean": Name_of_scie,
                        "URL": url2,
                        "Research Departments": dept_name ,
                        "Prgorams": all_programs
                        })
    #PSUT_Schools_info.append(school_info)
    PSUT_Staff_info.append(staff_name_w_position)



  print(school_info)

print(staff_name_w_position)


condition met


-----------------------Starting With King Abdullah II School of Engineering ----------------------------------------------






[{'School Name': 'King Abdullah II School of Engineering', 'Dean': 'Dr. Osama Abu-Sharkh', 'URL': 'https://psut.edu.jo/en/school/School_of_Engineering', 'Departments': ['Electrical Engineering Department', 'Basic Sciences Department', 'Communications Engineering Department', 'Computer Engineering Department'], 'Prgorams': ['Bachelor Degree in Communications Engineering \\ Internet of Things', 'Bachelor Degree in Electronics Engineering', 'Bachelor Degree in Computer Engineering', 'Bachelor Degree in Networks and Information Security Engineering', 'Bachelor Degree in Electrical Power and Energy Engineering', "Master's Degree in Electrical Engineering", "Master's Degree in Engineering Management"]}]
condition met


-----------------------Starting With King Hussein School of Computing Sciences ----------------------------------------------








In [80]:
print(school_info)

[{'School Name': 'King Abdullah I  School of Graduate Studies & Scientific Research', 'Dean': 'Prof. Raed Mesleh', 'URL': 'https://psut.edu.jo/en/school/School_of_Graduate_Studies_Scientific_Research', 'Research Departments': ['NLP Research Group', 'Data Science and Big Data Research Group', 'Security Research Group'], 'Prgorams': ["Master's Degree in Business Entrepreneurship", "Master's Degree in Business Analytics", "Master's Degree in E-Marketing and Social Media", "Master's Degree in Financial Technology and Accounting Analytics", "Master's Degree in Supply Chain Management and Logistics Technology", "Master's Degree in Computer Science", "Master's Degree in Data Science", "Master's Degree in Cyber security", "Master's Degree in Enterprise Systems Engineering", "Master's Degree in Health Information Technology", 'Doctorate of Philosophy in Computer Science', "Master's Degree in Electrical Engineering", "Master's Degree in Engineering Management"], 'Staff': ['Mrs. Ola Almahroq', 'M

In [100]:
#Store each in JSON files
import json
from google.colab import drive
drive.mount('/content/drive')
#All staff info
file_path = '/content/drive/My Drive/Colab Notebooks/school_info.json'

# Write the extracted data to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(school_info, json_file, indent=4)

print(f'Extracted data has been saved to {file_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted data has been saved to /content/drive/My Drive/Colab Notebooks/school_info.json


In [94]:
#Store each in JSON files
import json
from google.colab import drive
drive.mount('/content/drive')
#All staff info
file_path = '/content/drive/My Drive/Colab Notebooks/final_staff.json'

# Write the extracted data to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(final_staff, json_file, indent=4)

print(f'Extracted data has been saved to {file_path}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted data has been saved to /content/drive/My Drive/Colab Notebooks/final_staff.json


In [39]:
url5 = 'https://psut.edu.jo/en/school/School_of_Graduate_Studies_Scientific_Research'

response5 = requests.get(url5)
if response5.status_code == 200:
    html_content2 = response5.text

else:
    print(f"Failed to fetch the webpage. Status code: {response5.status_code}")

soup5 = BeautifulSoup(html_content2, 'html.parser')

#Scraping schools names, programs
school = soup5.find('h2')
school_name = school.text.strip()

scie_research_tab = soup5.find('div',  class_='row aboutgraduate trainingworkshop-box mt-1 school-custom-color')


depts = scie_research_tab.find_all('div',class_='related-items')

    #programe_links = programs.find_all('h4')
dept_name = [link.find('h2').text.strip() for link in depts]
print(dept_name)

['NLP Research Group', 'Data Science and Big Data Research Group', 'Security Research Group']


In [54]:
DeanName= soup5.find('div',class_='container padding-top-35')

Nameee = soup5.find('strong').get_text()
desired_text = Nameee.split('Research')
print(desired_text[-1])

Prof. Raed Mesleh
